In [3]:
import braket
from braket.circuits.gate import Gate as BraketGate
from braket.circuits.instruction import Instruction as BraketInstruction
import cirq
import qiskit
from qiskit.circuit import Parameter
import numpy as np
from qbraid.transpiler.transpiler import qbraid_wrapper

In [324]:
qiskit_circ = qiskit.QuantumCircuit(4)

qiskit_circ.h(0)
qiskit_circ.h(1)
qiskit_circ.h(2)
qiskit_circ.h(3)
qiskit_circ.x(0)
qiskit_circ.x(1)
qiskit_circ.y(2)
qiskit_circ.z(3)
qiskit_circ.s(0)
qiskit_circ.sdg(1)
qiskit_circ.t(2)
qiskit_circ.tdg(3)
qiskit_circ.rx(np.pi/4, 0)
qiskit_circ.ry(np.pi/2, 1)
qiskit_circ.rz(3*np.pi/4, 2)
qiskit_circ.p(np.pi/8, 3)
qiskit_circ.sx(0)
qiskit_circ.sxdg(1)
qiskit_circ.iswap(2, 3)
qiskit_circ.swap(0, 2)
qiskit_circ.swap(1, 3)
qiskit_circ.cx(0, 1)
qiskit_circ.cp(np.pi/4, 2, 3)

qiskit_circ.draw()

┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  ISWAP │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ TDG ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘

In [325]:
q0 = cirq.LineQubit(3)
q1 = cirq.LineQubit(2)
q2 = cirq.LineQubit(1)
q3 = cirq.LineQubit(0)

cirq_circ = cirq.Circuit()

cirq_circ.append(cirq.H(q0))
cirq_circ.append(cirq.H(q1))
cirq_circ.append(cirq.H(q2))
cirq_circ.append(cirq.H(q3))
cirq_circ.append(cirq.X(q0))
cirq_circ.append(cirq.X(q1))
cirq_circ.append(cirq.Y(q2))
cirq_circ.append(cirq.Z(q3))
cirq_circ.append(cirq.S(q0))
cirq_circ.append(cirq.ZPowGate(exponent=-0.5)(q1))       # Sdg
cirq_circ.append(cirq.T(q2))
cirq_circ.append(cirq.ZPowGate(exponent=-0.25)(q3))      # Tdg
cirq_circ.append(cirq.Rx(rads=np.pi/4)(q0))
cirq_circ.append(cirq.Ry(rads=np.pi/2)(q1))
cirq_circ.append(cirq.Rz(rads=3*np.pi/4)(q2))
cirq_circ.append(cirq.ZPowGate(exponent=1/8)(q3))        # Phase
cirq_circ.append(cirq.XPowGate(exponent=0.5)(q0))        # SX
cirq_circ.append(cirq.XPowGate(exponent=-0.5)(q1))       # SXdg
cirq_circ.append(cirq.ISWAP(q2, q3))
cirq_circ.append(cirq.SWAP(q0, q2))
cirq_circ.append(cirq.SWAP(q1, q3))
cirq_circ.append(cirq.CNOT(q0, q1))
cirq_circ.append(cirq.CZPowGate(exponent=0.25)(q2, q3))  # CPhase(pi/4)

print(cirq_circ)

                                          ┌──┐
0: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@────────
                                 │          │    │
1: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@^0.25───
                                           ││
2: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           │     │
3: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                          └──┘


In [326]:
cirq_unitary = cirq_circ.unitary()
qiskit_unitary = qiskit.quantum_info.Operator(qiskit_circ).data

In [327]:
np.allclose(cirq_unitary, qiskit_unitary)

True